In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.io import curdoc
import pandas as pd
import yaml
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column

In [ ]:
# Specified colors (corrected hex codes)
colors = ['#FE7F2D', '#619B8A', '#FCCA46', '#A1C181', '#233D4D']

In [ ]:
def create_plot(source_data):
    p = figure(plot_width=800, plot_height=250, x_axis_label='Weeks', y_axis_label='Value')
    p.line('weeks', 'value', source=source_data, line_width=2)
    return p

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

# Example DataFrames (load your data here)

def get_config():
    with open("config.yml", "r") as my_config:
        config = yaml.safe_load(my_config)
    return config
config = get_config()
        
df_before_ph = pd.read_csv(config['ph'])
df_before_plaque = pd.read_csv(config['plaque'])
df_before_gum_bleeding = pd.read_csv(config['gum_bleeding'])
df_lifesum = pd.read_csv(config['lifesum'])

df_after_ph = pd.read_csv(config['ph2'])
df_after_plaque = pd.read_csv(config['plaque2'])
df_after_gum_bleeding = pd.read_csv(config['gum_bleeding2'])
# Load your Plaque and Gum Bleeding dataframes similarly

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='factor-select',
        options=[
            {'label': 'PH', 'value': 'PH'},
            {'label': 'Plaque', 'value': 'Plaque'},
            {'label': 'Gum Bleeding', 'value': 'GumBleeding'}
        ],
        value='PH'  # Default value
    ),
    dcc.Graph(id='before-plot'),
    dcc.Graph(id='after-plot')
])

@app.callback(
    [Output('before-plot', 'figure'),
     Output('after-plot', 'figure')],
    [Input('factor-select', 'value')]
)
def update_graph(selected_factor):
    if selected_factor == 'PH':
        df_before = df_before_ph
        df_after = df_after_ph
    elif selected_factor == 'Plaque':
        # Assign df_before_plaque and df_after_plaque accordingly
        pass
    elif selected_factor == 'GumBleeding':
        # Assign df_before_gum_bleeding and df_after_gum_bleeding accordingly
        pass
    
    # Create traces for the "before" plot
    data_before = []
    for col in df_before.columns[1:]:  # Skip 'Weeks' column
        data_before.append(go.Scatter(x=df_before['Weeks'], y=df_before[col], mode='lines', name=col))
    
    # Create traces for the "after" plot
    data_after = []
    for col in df_after.columns[1:]:  # Skip 'Weeks' column
        data_after.append(go.Scatter(x=df_after['Weeks'], y=df_after[col], mode='lines', name=col))
    
    # Define the figure
    fig_before = {'data': data_before, 'layout': go.Layout(title=f'{selected_factor} Before')}
    fig_after = {'data': data_after, 'layout': go.Layout(title=f'{selected_factor} After')}
    
    return fig_before, fig_after

if __name__ == '__main__':
    app.run_server(debug=True)
